# Ethereum 2.0 Merge Proof-of-Concept

This notebook provides a proof-of-concept for the [Ethereum 2.0 Quick Merge Proposal](https://notes.ethereum.org/@vbuterin/B1mUf6DXO). A notable difference from the proposal is that Eth2 blocks in this PoC only contain Eth1 block hashes, instead of complete Eth1 blocks.

An unchanged Eth1 spec (from `py-evm`) has been used here.
The Eth2 spec has been slightly modified, and can be found in [this branch on `eth2.0-specs`](https://github.com/ethereum/eth2.0-specs/compare/adiasg-quick-merge-poc).

## Prerequisites
- Python3, Jupyter Notebook
- Run `make install` to:
    - create a venv.
    - download and install Eth1 & Eth2 python modules in the venv.
    - install this repo as a local package in the venv.
    - install the venv as a kernel (named `merge-spec-poc`) for Jupyter notebook.
- Choose the `merge-spec-poc` kernel to execute this Jupyter notebook.

## Other Merge Specification Efforts
- Mikhail Kalinin: https://github.com/ethereum/eth2.0-specs/pull/2257

In [1]:
from eth1 import Eth1Rpc
from eth.chains.base import MiningChain, Chain
from eth.vm.forks import IstanbulVM
import eth.tools.builder.chain as builder
        
# Initialize Eth1 chain builder
Eth1Chain = builder.build(MiningChain, builder.fork_at(IstanbulVM, 0))
Eth1Chain = builder.enable_pow_mining(Eth1Chain)
eth1_rpc = Eth1Rpc(Eth1Chain)

# Build a short Eth1 chain
eth1_blocks = [eth1_rpc.consensus_assembleBlock() for i in range(9)]
eth1_rpc.print_block(eth1_blocks[-1])

Eth1 Block #9
	Parent Hash:	0x72ef2df3c13e504f97b5be521e958daf9f62e2e0692b3223c9cba2f211406587
	Block Hash:	0x2ba0808e07a82deea42a1d5383c72faf83204b407d0ebbcb21291532dcfb04f0
	Block Score:	10


In [2]:
import eth2spec.phase0.spec as spec

# Fill in the Eth1 RPC functions into the Eth2 state transition logic

def _is_valid_eth1_block(eth1_block_hash: spec.Bytes32) -> bool:
    assert eth1_rpc.is_accepted_block(eth1_block_hash)
    assert eth1_rpc.get_score(eth1_block_hash) >= spec.TRANSITION_TOTAL_DIFFICULTY
    return True

def _is_eth1_parent_block(parent_block_hash: spec.Bytes32, current_block_hash: spec.Bytes32) -> bool:
    assert eth1_rpc.is_parent_block(parent_block_hash, current_block_hash)
    return True

spec.is_valid_eth1_block = _is_valid_eth1_block
spec.is_eth1_parent_block = _is_eth1_parent_block

In [3]:
# Eth2 chain building tools
from eth2spec.test.helpers import genesis

genesis_state = genesis.create_genesis_state(spec, [32*10**9 for i in range(100)], 16*10**9)
genesis_block = spec.BeaconBlock(state_root=genesis_state.hash_tree_root())
store = spec.get_forkchoice_store(genesis_state, genesis_block)

In [4]:
from eth2spec.test.helpers.block import build_empty_block_for_next_slot, sign_block
from eth2spec.test.helpers.state import state_transition_and_sign_block, next_epoch

# Push the Eth2 state to some arbitraty slot
time = spec.SECONDS_PER_SLOT * 1000
spec.on_tick(store, time)
state = genesis_state.copy()
next_epoch(spec, state)

# Build a short Eth2 chain
for i in range(5):
    block = build_empty_block_for_next_slot(spec, state)
    block.body.application_block_hash = spec.ZERO_HASH
    signed_block = state_transition_and_sign_block(spec, state, block)

def print_eth2_state(state):
    print("Eth2 State:")
    print(f"\tSlot: {state.slot}")
    print(f"\tPrev. Eth1 Block: {state.previous_application_block_hash}")
    if state.previous_application_block_hash != spec.ZERO_HASH:
        eth1_rpc.print_block(eth1_rpc.chain.get_block_by_hash(state.previous_application_block_hash), "\t")

# This is the Eth2 state before the merge
print_eth2_state(state)

Eth2 State:
	Slot: 37
	Prev. Eth1 Block: 0x0000000000000000000000000000000000000000000000000000000000000000


In [5]:
# TRANSITION_TOTAL_DIFFICULTY is set to 10
# eth1_blocks[7] is the last PoW block
# eth1_blocks[8] is the first transition block

block = build_empty_block_for_next_slot(spec, state)
# Fill in the last PoW mined block
block.body.application_block_hash = eth1_blocks[8].hash
signed_block = state_transition_and_sign_block(spec, state, block)
print_eth2_state(state)

Eth2 State:
	Slot: 38
	Prev. Eth1 Block: 0x2ba0808e07a82deea42a1d5383c72faf83204b407d0ebbcb21291532dcfb04f0
	Eth1 Block #9
		Parent Hash:	0x72ef2df3c13e504f97b5be521e958daf9f62e2e0692b3223c9cba2f211406587
		Block Hash:	0x2ba0808e07a82deea42a1d5383c72faf83204b407d0ebbcb21291532dcfb04f0
		Block Score:	10


## The Merge has happened!
## The last PoW block has been included in the Eth2 chain.
## All future Eth1 blocks will be produced by Eth2 PoS validator.

In [6]:
# Progress the Eth2 chain after the merge
for i in range(5):
    block = build_empty_block_for_next_slot(spec, state)
    # The eth1_rpc.mine_block() call in the next line is run by the Eth2 validator on its Eth1 node.
    # These blocks are not PoW intensive -- their difficulty will be 1.
    block.body.application_block_hash = eth1_rpc.consensus_assembleBlock().hash
    signed_block = state_transition_and_sign_block(spec, state, block)
    print_eth2_state(state)

Eth2 State:
	Slot: 39
	Prev. Eth1 Block: 0x71551c99521d285c0f2687930b3f2e5c2f0dee3e309a409df6b7dad4fe51c824
	Eth1 Block #10
		Parent Hash:	0x2ba0808e07a82deea42a1d5383c72faf83204b407d0ebbcb21291532dcfb04f0
		Block Hash:	0x71551c99521d285c0f2687930b3f2e5c2f0dee3e309a409df6b7dad4fe51c824
		Block Score:	11
Eth2 State:
	Slot: 40
	Prev. Eth1 Block: 0x20dcee87d415c181fdb892d7b3737f13257b266766939477ef90caede7534d05
	Eth1 Block #11
		Parent Hash:	0x71551c99521d285c0f2687930b3f2e5c2f0dee3e309a409df6b7dad4fe51c824
		Block Hash:	0x20dcee87d415c181fdb892d7b3737f13257b266766939477ef90caede7534d05
		Block Score:	12
Eth2 State:
	Slot: 41
	Prev. Eth1 Block: 0x7d47d62ba5fa9343b3c48c10deecb43ab1f5babfdb7e856060e80a6e45b94b60
	Eth1 Block #12
		Parent Hash:	0x20dcee87d415c181fdb892d7b3737f13257b266766939477ef90caede7534d05
		Block Hash:	0x7d47d62ba5fa9343b3c48c10deecb43ab1f5babfdb7e856060e80a6e45b94b60
		Block Score:	13
Eth2 State:
	Slot: 42
	Prev. Eth1 Block: 0x0dc4d00ff7f7bd810846a98c4d9bceccb8ee14ed4a